In [2]:
months = ['may']

In [55]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
from fuzzywuzzy import process, fuzz
import numpy as np
from math import ceil

In [37]:
from szp_funcs import load_month_all, custom_create_res, print_df, get_fio, get_otv, gku_inn, load_groups, day_types, path_to_data, path_to_docs

In [16]:
ppl = load_month_all(months[-1])
ped, isp, aup = load_groups('input')
ppl[ppl['type_' + months[-1]] == day_types[1]]

,inn,snils,sum_may,job_may,day_may,type_may,stv_may,status_may,status_pref_may,wday_may,nday_may


In [17]:
ppl[ppl.snils == '033-839-515 65']

,inn,snils,sum_may,job_may,day_may,type_may,stv_may,status_may,status_pref_may,wday_may,nday_may
53996,7717043346,033-839-515 65,205795.0,Ведущий научный сотрудник,1,Основное место работы,1.0,Работа,Работа,18.0,18.0


In [24]:
ans = ppl[(ppl['sum_' + months[-1]] > 200000) & (ppl['inn'].isin(gku_inn) == False) & (ppl['job_' + months[-1]].isin(aup + ped) == False) & (ppl['status_' + months[-1]].isin(['Увольнение']) == False) & (ppl['type_' + months[-1]] == day_types[0])]

In [25]:
ans

,inn,snils,sum_may,job_may,day_may,type_may,stv_may,status_may,status_pref_may,wday_may,nday_may
655,5003021368,143-108-314 14,213060.00,Ведущий экономист,1,Основное место работы,1.0,Работа,Работа,18.0,18.0
1508,5003021495,106-994-733 95,212768.00,Ведущий экономист,1,Основное место работы,1.0,Работа,Работа,18.0,18.0
2486,5003021640,075-365-019 69,218583.34,Ведущий экономист,0,Основное место работы,1.0,Работа,Работа,15.0,18.0
3327,5003096290,059-350-413 62,295015.00,Ведущий экономист,1,Основное место работы,0.9,Работа в отпуске по уходу за ребенком,Работа в отпуске по уходу за ребенком,18.0,18.0
4817,5051005503,046-964-438 99,204857.28,Ведущий экономист,1,Основное место работы,1.0,Работа,Работа,18.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...
191582,7743085744,127-026-731 35,248368.08,Заведующий отделением,1,Основное место работы,1.0,Работа,Работа,18.0,18.0
192615,7743905443,021-946-435 43,326032.80,Начальник службы,1,Основное место работы,1.0,Работа,Работа,18.0,18.0
193104,7743905443,127-699-045 02,258720.38,Начальник отдела кадров,1,Основное место работы,1.0,Работа,Работа,18.0,18.0
194464,7743937357,026-884-480 89,280000.00,Начальник отдела,1,Основное место работы,1.0,Работа,Работа,18.0,18.0


In [34]:
print_df(ans, 'ans ' + months[-1])

In [26]:
otv = get_otv()
otv = otv.drop('ekis', axis=1)

In [27]:
otv

,inn,okr,otv,sch_name
0,7743085543,САО,Рагимова Виктория Олеговна,ГБОУ Спецшкола № 1
1,7713229928,САО,Горбанева Светлана Васильевна,ГКОУ КШИ № 1
2,7704271031,ЦАО,Майильян Елена Александровна,ГБПОУ ПТ № 2
3,7714093099,САО,Горбанева Светлана Васильевна,ГКОУ СКОШИ № 2
4,7709438340,ЦАО,Майильян Елена Александровна,ГБПОУ КСУ № 3
...,...,...,...,...
935,7728113806,NaN,NaN,ИКИ РАН
936,7728013512,NaN,NaN,ИПУ РАН
937,7717067481,NaN,NaN,ГБОУ ШОР № 1
938,7729133548,NaN,NaN,СУНЦ МГУ


In [20]:
fio = get_fio()
fio

,snils,fio
0,001-879-913 57,Солодовникова Виктория Дмитриевна
1,001-896-420 39,Серегина Ольга Васильевна
2,002-050-383 67,Шепелева Ольга Алексеевна
3,002-065-271 85,Галицина Ольга Викторовна
4,002-845-525 25,Рубцова Ирина Ивановна
...,...,...
184799,193-592-846 21,Бокова Вероника Геннадьевна
184800,161-301-903 15,Филатов Иван Владимирович
184801,057-135-721 55,Гуляева Анна Александровна
184802,190-942-402 76,Ильющенков Александр Владимирович


In [29]:
pretty_ppl = pd.merge(ans, fio, how='left', on='snils')
pretty_ppl = pd.merge(pretty_ppl, otv, how='left', on='inn')
pretty_ppl = pretty_ppl.drop_duplicates(['snils'])

In [30]:
pretty_ppl

,inn,snils,sum_may,job_may,day_may,type_may,stv_may,status_may,status_pref_may,wday_may,nday_may,fio,okr,otv,sch_name
0,5003021368,143-108-314 14,213060.00,Ведущий экономист,1,Основное место работы,1.0,Работа,Работа,18.0,18.0,Шеламова Юлия Сергеевна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 1788
1,5003021495,106-994-733 95,212768.00,Ведущий экономист,1,Основное место работы,1.0,Работа,Работа,18.0,18.0,Кваснюк Анастасия Васильевна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 2070
2,5003021640,075-365-019 69,218583.34,Ведущий экономист,0,Основное место работы,1.0,Работа,Работа,15.0,18.0,Васина Виктория Викторовна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 1392 им. Д.В. Рябинкина
3,5003096290,059-350-413 62,295015.00,Ведущий экономист,1,Основное место работы,0.9,Работа в отпуске по уходу за ребенком,Работа в отпуске по уходу за ребенком,18.0,18.0,Пестова Юлия Владимировна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 2065
4,5051005503,046-964-438 99,204857.28,Ведущий экономист,1,Основное место работы,1.0,Работа,Работа,18.0,18.0,Рогова Элла Анатольевна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 2117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,7743085744,127-026-731 35,248368.08,Заведующий отделением,1,Основное место работы,1.0,Работа,Работа,18.0,18.0,Стебунов Сергей Викторович,САО,Рагимова Виктория Олеговна,ГБПОУ ТПСК им. В. М. Максимчука
432,7743905443,021-946-435 43,326032.80,Начальник службы,1,Основное место работы,1.0,Работа,Работа,18.0,18.0,Варламова Надежда Викторовна,САО,Рагимова Виктория Олеговна,ГБОУ Школа № 2098 имени Героя Советского Союза...
433,7743905443,127-699-045 02,258720.38,Начальник отдела кадров,1,Основное место работы,1.0,Работа,Работа,18.0,18.0,Кочнева Любовь Вячеславовна,САО,Рагимова Виктория Олеговна,ГБОУ Школа № 2098 имени Героя Советского Союза...
434,7743937357,026-884-480 89,280000.00,Начальник отдела,1,Основное место работы,1.0,Работа,Работа,18.0,18.0,Алексашин Александр Леонидович,САО,Рагимова Виктория Олеговна,ГБПОУ ПК им. Н.Н. Годовикова


In [33]:
print_df(pretty_ppl[['sch_name', 'okr', 'inn', 'fio', 'job_' + months[-1], 'sum_' + months[-1], 'stv_' + months[-1], 'otv', 'snils']], '200+_' + months[-1] + '_cute')

In [ ]:
aup

['Главный бухгалтер',
 'Директор',
 'Директор филиала',
 'Заведующий',
 'Заместитель главного бухгалтера',
 'Заместитель директора',
 'Заместитель начальника',
 'Заместитель руководителя',
 'Начальник',
 'Первый заместитель начальника',
 'Первый заместитель руководителя',
 'Первый заместитель директора',
 'Заместитель начальника - начальник управления',
 'Первый проректор',
 'Президент',
 'Проректор',
 'Ректор',
 'Руководитель']

In [36]:
full_res = pd.read_excel(path_to_data + months[-1] + '.xlsx')
nach_res = pd.read_excel(path_to_data + months[-1] + '_nach.xlsx')

In [38]:
nach_grps = pd.read_excel(path_to_docs + 'pay_type.xlsx', sheet_name='Все').rename(columns={'Вид ': 'type', 'Наименование ': 'name'})[['type', 'name']]

In [39]:
snils_of_int = ans['snils'].unique().tolist()

In [48]:
t_ans = ans.rename(columns={
    'sum_may': 'sum',
    'type_may': 'type',
    'job_may': 'job'
})[['inn', 'snils']]

In [49]:
t_ans

,inn,snils
655,5003021368,143-108-314 14
1508,5003021495,106-994-733 95
2486,5003021640,075-365-019 69
3327,5003096290,059-350-413 62
4817,5051005503,046-964-438 99
...,...,...
191582,7743085744,127-026-731 35
192615,7743905443,021-946-435 43
193104,7743905443,127-699-045 02
194464,7743937357,026-884-480 89


In [52]:
int_full_res = full_res[full_res['snils'].isin(snils_of_int)][['inn', 'snils', 'sum', 'job', 'stv', 'type']]
int_nach_res = nach_res[nach_res['snils'].isin(snils_of_int)]

In [53]:
int_full_res

,inn,snils,sum,job,stv,type
900,7736076587,042-790-661 63,202869.23,Ведущий специалист,"1,00000000000000000000",Основное место работы
1528,7717043346,001-354-128 81,180712.47,Начальник отдела,"1,00000000000000000000",Основное место работы
1529,7717043346,001-354-128 81,11792.65,Преподаватель,"0,30000000000000000000",Внутреннее совместительство
1530,7717043346,001-354-128 81,67041.16,Преподаватель,"0,20000000000000000000",Внутреннее совместительство
1532,7717043346,001-370-187 86,122876.12,Заведующий лабораторией,"1,00000000000000000000",Основное место работы
...,...,...,...,...,...,...
192463,7724295383,020-735-443 15,228100.00,Экономист,"1,00000000000000000000",Основное место работы
192813,7743085744,127-026-731 35,248368.08,Заведующий отделением,"1,00000000000000000000",Основное место работы
194451,7713229928,128-380-296 68,253124.85,Администратор,"1,00000000000000000000",Основное место работы
194482,7713229928,134-787-105 78,286891.69,Администратор,"1,00000000000000000000",Основное место работы


In [54]:
int_nach_res

,inn,snils,nach_type,nach_name,nach_sum
4065,7736076587,042-790-661 63,Основное место работы,Выплата за результативность работы в предыдуще...,26650.60
4066,7736076587,042-790-661 63,Основное место работы,Доплата за увеличение объема работ,61167.50
4067,7736076587,042-790-661 63,Основное место работы,Оплата по окладу,108348.00
4068,7736076587,042-790-661 63,Основное место работы,"Премия за результаты работы, выразившиеся в о...",6703.13
6993,7717043346,001-354-128 81,Основное место работы,Выплата стимулирующего характера за качественн...,85000.00
...,...,...,...,...,...
880045,7713229928,134-787-105 78,Основное место работы,Премия за подготовку и проведение важных мероп...,50000.00
880392,7713229928,163-706-884 87,Основное место работы,Выплата за результативность работы в предыдуще...,30000.00
880393,7713229928,163-706-884 87,Основное место работы,Доплата за исполнение обязанностей временно от...,13137.42
880394,7713229928,163-706-884 87,Основное место работы,Доплата за работу в выходные и праздничные дни,17466.67


In [56]:
def matcher(row, look_up, name):
    ans = process.extractOne(row[name], look_up, scorer=fuzz.WRatio, score_cutoff=60)
    if ans:
        return ans[0]
    return ans

In [61]:
nach_grps

,type,name
0,Базовая часть,Оклад
1,Базовая часть,Должностной оклад
2,Базовая часть,Ставка заработной платы
3,Базовая часть,Тарифная ставка
4,Компенсационные выплаты,Доплата за совмещение профессий (должностей)
5,Компенсационные выплаты,Доплата за расширение зон обслуживания
6,Компенсационные выплаты,Доплата за увеличение объема работ
7,Компенсационные выплаты,Доплата за выполнение обязанностей временно от...
8,Компенсационные выплаты,Доплата за работу в выходные и праздничные дни
9,Компенсационные выплаты,Доплата за работу в ночное время


In [59]:
namer = pd.DataFrame(int_nach_res['nach_name'].unique(), columns=['nach_name'])
namer['new_name'] = namer.apply(lambda row: matcher(row, nach_grps['name'].to_list(), 'nach_name'), axis=1)
namer.fillna('Не преведено', inplace=True)
namer = pd.merge(namer, nach_grps, how='left', left_on='new_name', right_on='name').drop('name',axis=1).fillna('Не преведено')

In [108]:
namer[namer.type == 'Не преведено']

,nach_name,new_name,type
11,ВСХ-МЕР-21 Научно-методическое обеспечение соп...,Не преведено,Не преведено
15,Контракт № ВЕМР/АF/2.5/1-3-3 от 15 декабря 202...,Не преведено,Не преведено
19,"Соглашение № 075-15-2021-1302/1 программа ""При...",Не преведено,Не преведено
20,ВСХ Лучший инициативный проект МГПУ-2021,Не преведено,Не преведено
115,Компенсация отпуска (Ежегодный дополнительный ...,Не преведено,Не преведено
143,Персональный коэффициент Кпер,Не преведено,Не преведено
144,Персональный коэффициент Кпер внебюджет,Не преведено,Не преведено
146,Часы Педаг.Дополнит.Образов. (бюджет),Не преведено,Не преведено
162,Консультации бюджет,Не преведено,Не преведено
163,Консультации внебюджет,Не преведено,Не преведено


In [92]:
t_nach = pd.merge(int_nach_res, namer, how='left').drop(['nach_name', 'new_name'], axis=1).groupby(['inn', 'snils', 'nach_type', 'type']).sum().unstack()
t_nach.columns = [i[1] for i in t_nach.columns]
t_nach.reset_index(inplace=True)

In [93]:
t_nach

,inn,snils,nach_type,Базовая часть,Компенсационные выплаты,Не преведено,Социальные выплаты и гарантии,Стимулирующие выплаты
0,5003021368,143-108-314 14,Основное место работы,213060.00,NaN,NaN,NaN,NaN
1,5003021495,106-994-733 95,Внутреннее совместительство,38813.50,38813.50,NaN,NaN,NaN
2,5003021495,106-994-733 95,Основное место работы,87641.00,NaN,NaN,NaN,47500.00
3,5003021640,075-365-019 69,Основное место работы,176416.67,42166.67,NaN,NaN,NaN
4,5003096290,059-350-413 62,Основное место работы,188915.00,NaN,NaN,NaN,106100.00
...,...,...,...,...,...,...,...,...
509,7743905443,127-699-045 02,Внутреннее совместительство,22585.32,5250.00,NaN,NaN,4517.06
510,7743905443,127-699-045 02,Основное место работы,188640.00,NaN,NaN,NaN,37728.00
511,7743937357,026-884-480 89,Основное место работы,200000.00,NaN,NaN,NaN,80000.00
512,9715206976,056-748-271 96,Внутреннее совместительство,40000.00,NaN,NaN,NaN,NaN


In [96]:
big_ans = pd.merge(int_full_res, t_nach, right_on=['inn', 'snils', 'nach_type'], left_on=['inn', 'snils', 'type'], how='left').drop('nach_type', axis=1)

In [104]:
big_ans

,inn,snils,sum,job,stv,type,Базовая часть,Компенсационные выплаты,Не преведено,Социальные выплаты и гарантии,Стимулирующие выплаты
0,7736076587,042-790-661 63,202869.23,Ведущий специалист,"1,00000000000000000000",Основное место работы,108348.00,61167.50,NaN,NaN,33353.73
1,7717043346,001-354-128 81,180712.47,Начальник отдела,"1,00000000000000000000",Основное место работы,64241.11,85000.00,NaN,90599.04,25000.00
2,7717043346,001-354-128 81,11792.65,Преподаватель,"0,30000000000000000000",Внутреннее совместительство,18915.22,23315.07,NaN,42369.28,NaN
3,7717043346,001-354-128 81,67041.16,Преподаватель,"0,20000000000000000000",Внутреннее совместительство,18915.22,23315.07,NaN,42369.28,NaN
4,7717043346,001-370-187 86,122876.12,Заведующий лабораторией,"1,00000000000000000000",Основное место работы,48966.67,NaN,NaN,23909.45,50000.00
...,...,...,...,...,...,...,...,...,...,...,...
516,7724295383,020-735-443 15,228100.00,Экономист,"1,00000000000000000000",Основное место работы,135600.00,82500.00,NaN,NaN,10000.00
517,7743085744,127-026-731 35,248368.08,Заведующий отделением,"1,00000000000000000000",Основное место работы,86942.08,40386.00,NaN,NaN,121040.00
518,7713229928,128-380-296 68,253124.85,Администратор,"1,00000000000000000000",Основное место работы,157200.00,45924.85,NaN,NaN,50000.00
519,7713229928,134-787-105 78,286891.69,Администратор,"1,00000000000000000000",Основное место работы,157200.00,79691.69,NaN,NaN,50000.00


In [105]:
pretty_big_ans = pd.merge(big_ans, fio, how='left', on='snils')

In [106]:
pretty_big_ans = pd.merge(pretty_big_ans, otv, how='left', on='inn')

In [112]:
pretty_big_ans

,inn,snils,sum,job,stv,type,Базовая часть,Компенсационные выплаты,Не преведено,Социальные выплаты и гарантии,Стимулирующие выплаты,fio,okr,otv,sch_name
0,7736076587,042-790-661 63,202869.23,Ведущий специалист,"1,00000000000000000000",Основное место работы,108348.00,61167.50,NaN,NaN,33353.73,Валяшко Мария Георгиевна,ЮЗАО,Застаева Таисия Саналовна,ГБОУ Школа № 1514
1,7717043346,001-354-128 81,180712.47,Начальник отдела,"1,00000000000000000000",Основное место работы,64241.11,85000.00,NaN,90599.04,25000.00,Крюкова Наталья Юрьевна,СВАО,Горбанева Светлана Васильевна,ГАОУ ВО МГПУ
2,7717043346,001-354-128 81,11792.65,Преподаватель,"0,30000000000000000000",Внутреннее совместительство,18915.22,23315.07,NaN,42369.28,NaN,Крюкова Наталья Юрьевна,СВАО,Горбанева Светлана Васильевна,ГАОУ ВО МГПУ
3,7717043346,001-354-128 81,67041.16,Преподаватель,"0,20000000000000000000",Внутреннее совместительство,18915.22,23315.07,NaN,42369.28,NaN,Крюкова Наталья Юрьевна,СВАО,Горбанева Светлана Васильевна,ГАОУ ВО МГПУ
4,7717043346,001-370-187 86,122876.12,Заведующий лабораторией,"1,00000000000000000000",Основное место работы,48966.67,NaN,NaN,23909.45,50000.00,Иоффе Андрей Наумович,СВАО,Горбанева Светлана Васильевна,ГАОУ ВО МГПУ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,7724295383,020-735-443 15,228100.00,Экономист,"1,00000000000000000000",Основное место работы,135600.00,82500.00,NaN,NaN,10000.00,Артемова Галина Романовна,ЮАО,Рагимова Виктория Олеговна,ГБПОУ ТК № 34
517,7743085744,127-026-731 35,248368.08,Заведующий отделением,"1,00000000000000000000",Основное место работы,86942.08,40386.00,NaN,NaN,121040.00,Стебунов Сергей Викторович,САО,Рагимова Виктория Олеговна,ГБПОУ ТПСК им. В. М. Максимчука
518,7713229928,128-380-296 68,253124.85,Администратор,"1,00000000000000000000",Основное место работы,157200.00,45924.85,NaN,NaN,50000.00,Воронина Надежда Евгеньевна,САО,Горбанева Светлана Васильевна,ГКОУ КШИ № 1
519,7713229928,134-787-105 78,286891.69,Администратор,"1,00000000000000000000",Основное место работы,157200.00,79691.69,NaN,NaN,50000.00,Павлющенко Александр Михайлович,САО,Горбанева Светлана Васильевна,ГКОУ КШИ № 1


In [113]:
pd.merge(t_ans, pretty_big_ans, how='left')

,inn,snils,sum,job,stv,type,Базовая часть,Компенсационные выплаты,Не преведено,Социальные выплаты и гарантии,Стимулирующие выплаты,fio,okr,otv,sch_name
0,5003021368,143-108-314 14,213060.00,Ведущий экономист,"1,00000000000000000000",Основное место работы,213060.00,NaN,NaN,NaN,NaN,Шеламова Юлия Сергеевна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 1788
1,5003021495,106-994-733 95,135141.00,Ведущий экономист,"1,00000000000000000000",Основное место работы,87641.00,NaN,NaN,NaN,47500.00,Кваснюк Анастасия Васильевна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 2070
2,5003021495,106-994-733 95,77627.00,Экономист,"0,50000000000000000000",Внутреннее совместительство,38813.50,38813.50,NaN,NaN,NaN,Кваснюк Анастасия Васильевна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 2070
3,5003021640,075-365-019 69,218583.34,Ведущий экономист,"1,00000000000000000000",Основное место работы,176416.67,42166.67,NaN,NaN,NaN,Васина Виктория Викторовна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 1392 им. Д.В. Рябинкина
4,5003096290,059-350-413 62,295015.00,Ведущий экономист,"0,90000000000000000000",Основное место работы,188915.00,NaN,NaN,NaN,106100.00,Пестова Юлия Владимировна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 2065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,7743905443,127-699-045 02,226368.00,Начальник отдела кадров,"1,00000000000000000000",Основное место работы,188640.00,NaN,NaN,NaN,37728.00,Кочнева Любовь Вячеславовна,САО,Рагимова Виктория Олеговна,ГБОУ Школа № 2098 имени Героя Советского Союза...
507,7743905443,127-699-045 02,32352.38,Учитель,"0,44000000000000000000",Внутреннее совместительство,22585.32,5250.00,NaN,NaN,4517.06,Кочнева Любовь Вячеславовна,САО,Рагимова Виктория Олеговна,ГБОУ Школа № 2098 имени Героя Советского Союза...
508,7743937357,026-884-480 89,280000.00,Начальник отдела,"1,00000000000000000000",Основное место работы,200000.00,NaN,NaN,NaN,80000.00,Алексашин Александр Леонидович,САО,Рагимова Виктория Олеговна,ГБПОУ ПК им. Н.Н. Годовикова
509,9715206976,056-748-271 96,169219.37,Заведующий учебной частью,"1,00000000000000000000",Основное место работы,100000.00,69219.37,NaN,NaN,NaN,Пьянкова Татьяна Александровна,СВАО,Горбанева Светлана Васильевна,ГБОУ Школа Марьина Роща


In [109]:
print_df(pretty_big_ans, '200+ с начилениями ' + months[-1])